In [2]:
# importing libraries

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random
import time
from bs4 import BeautifulSoup
import bs4
from selenium.common.exceptions import TimeoutException
import pandas as pd
import os

In [18]:
url = 'https://www.glassdoor.co.in/Job/india-data-analyst-jobs-SRCH_IL.0,5_IN115_KO6,18.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data%2520&typedLocation=India&context=Jobs&dropdown=0'
driver = webdriver.Chrome()
driver.get(url)

# create empty hrml list

html_list = []

# initialize the number of elements loaded to be zero

num_elements_loaded = 0

# intialising the loop

while True:
    timeout = random.randint(2, 10) # seting a random value for the timeout for every iteration

    # The num_elements variable is used to count the number of elements that have been loaded.

    num_elements = len(driver.find_elements(by=By.CSS_SELECTOR, value=".d-flex.css-152bcv1.exy0tjh3"))
    num_elements_loaded += num_elements # add the len of num_elemment to number of elements loaded in every iteraion
    print(f'number of elements found: {num_elements_loaded}')

    # If 500 elements are loaded, break the loop

    if num_elements_loaded >= 5:
        print(f'Elements loaded {num_elements_loaded}, breaking the loop.')
        break

    # # Wait for a random amount of time before reloading more elements.

    time.sleep(timeout) 
    
    # Scroll down to load more elements

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for more elements to load

    try:
        WebDriverWait(driver, 1000).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".d-flex.css-152bcv1.exy0tjh3")))
    except TimeoutException:
        # If timeout occurs, increase the timeout and continue scrolling

        continue

    # Store the HTML of the current scroll in a list

    html_list.append(driver.page_source)

# quit the driver

driver.quit()

number of elements found: 1
number of elements found: 2
number of elements found: 3
number of elements found: 4
number of elements found: 5
Elements loaded 5, breaking the loop.


In [19]:
for i, html in enumerate(html_list):
    # Parse the HTML string using BeautifulSoup
    soup = BeautifulSoup(html, 'html.parser')
    # Write the prettified HTML to a file with a unique filename
    with open(f'output{i}.html', 'w', encoding='utf-8') as file:
        file.write(soup.prettify())

In [20]:
import datetime

# Define the extraction functions
def text_extractor(node):
    return node.text

def string_extractor(node):
    if type(node.contents[0]) is bs4.element.NavigableString:
        return node.contents[0]
    else:
        return None

# Define a new function that includes the current date and time
def datetime_extractor(node):
    now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
    if type(node.contents[0]) is bs4.element.NavigableString:
        return f"{node.contents[0]} ({now})"
    else:
        return None


In [6]:
# Define the extraction functions

def text_extractor(node):
    return node.text
def string_extractor(node):
    if type(node.contents[0]) is bs4.element.NavigableString:
        return node.contents[0]
    else:
        return None

In [36]:
# create classes dictionary

classes = {
    'rating': {
        'element': 'span',
        'cls': 'css-2lqh28 e1cjmv6j1',
        'extr': string_extractor
    },
    'company_name': {
        'element': 'a',
        'cls': 'css-l2wjgv e1n63ojh0 jobLink',
        'extr': text_extractor
    },
    'job_position': {
        'element': 'a',
        'cls': 'jobLink css-1rd3saf eigr9kq2',
        'extr': text_extractor
    },
    'location': {
        'element': 'span',
        'cls': 'css-3g3psg pr-xxsm css-iii9i8 e1rrn5ka0',
        'extr': text_extractor
    },
    'salary_estimate': {
        'element': 'span',
        'cls': 'css-1xe2xww e1wijj242',
        'extr': text_extractor
    },
    'date_time': {
        'element': 'div',
        'cls': 'd-flex align-items-end pl-std css-1vfumx3',
        'extr': datetime_extractor
    }
}

In [37]:
# Create an empty list to store the extracted data
data = []

# Loop through the html files and extract data from each file
for i in range(1):
    filename = f'output{i}.html'
    with open(filename, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'html.parser')
    result = {}

    # Loop through the classes dictionary and extract data for each class
    for key in classes:
        collection = soup.find_all(classes[key]['element'], class_=classes[key]['cls'])
        if collection:
            result[key] = [classes[key]['extr'](c) for c in collection]
        else:
            result[key] = [None] * len(data) # Fill in missing values with None

    # Ensure that the length of the extracted data is consistent for all classes
    max_len = max([len(result[key]) for key in result])
    for key in result:
        if len(result[key]) < max_len:
            result[key] += [None] * (max_len - len(result[key]))

    # result['file'] = filename # Add a new column to indicate which file the data is from
    data.append(result) # Append the result dictionary to the data list

# Concatenate the data from all files into a single DataFrame
dataframe = pd.concat([pd.DataFrame(d) for d in data], ignore_index=True)


In [38]:
# replace \n from the dataframe

dataframe = dataframe.replace('\n', '', regex=True)
dataframe.reset_index()

,index,rating,company_name,job_position,location,salary_estimate,date_time
0,0,4.1,S&P Global,Data Analyst,India,₹3L - ₹6L ...,24h (2023-04-...
1,1,4.2,Airtel India,Data Analyst,Gurgaon,₹5L - ₹9L ...,30d+ (2023-04...
2,2,4.0,Barclays,Analyst - RFT Data Operations ...,New Delhi,₹40T - ₹45T ...,6d (2023-04-0...
3,3,4.2,Glamyo Health,Data Analyst,Connaught Place,₹8L - ₹8L ...,15d (2023-04-...
4,4,5.0,Airtel India,Business Analyst - Messaging ...,Gurgaon,₹7L - ₹11L ...,1 day ago (20...
5,5,3.2,Fincor Management Advisory Se...,Data engineering ESG Analyst ...,Chandigarh,₹10T - ₹30T ...,17d (2023-04-...
6,6,4.0,Simply logistics pvt ltd ...,Data Analyst,New Delhi,₹55T - ₹70T ...,4d (2023-04-0...
7,7,5.0,Dileep Essentials Pvt. Ltd. ...,Data Analyst,Jaipur,₹5L - ₹10L ...,4d (2023-04-0...
8,8,3.8,RxLogix,Data Analyst,Noida,₹5L - ₹8L ...,3d (2023-04-0...
9,9,3.0,Barclays,Analyst - Impairment Analytics...,New Delhi,₹3L - ₹6L ...,6d (2023-04-0...
